<a href="https://colab.research.google.com/github/Existanze54/sirius-neural-networks-2024/blob/main/Homeworks/03HW_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 3. Оптимизация моделей и Transfer learning
В этом задании вы будете работать с датасетом [DermaMNIST](https://medmnist.com/). Используя материалы семинара 5 (05S_NN_methods), необходимо будет обучить две простые модели, а затем создать свою собственную, призванную побить их по метрикам качества.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
if torch.cuda.is_available():
  print(torch.cuda.get_device_properties(device))

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)

# Задание 0. Подготовка данных
Подгрузите датасет DermaMNIST, оформите даталоадеры, задайте функцию потерь.

In [ ]:
pip install --upgrade git+https://github.com/MedMNIST/MedMNIST.git

In [ ]:
import medmnist
from medmnist import INFO, Evaluator

In [ ]:
DataClass = # your code here

Выберем лосс функцию исходя из задачи


In [ ]:
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()
criterion

In [ ]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
train_dataset = # your code here
test_dataset = # your code here

BATCH_SIZE = 128 # оставим таким на протяжении всего ноутбука

train_loader = # your code here
train_loader_at_eval = # your code here
test_loader = # your code here

In [ ]:
print(train_dataset)
print("===================")
print(test_dataset)

In [ ]:
train_dataset.montage(length=10)

# Задание 1. Простая модель
Попробуем создать простую нейросеть с fully connected структурой и обучить ее.

Реализуйте следующую архитектуру:
1. Flatten слой
2. Линейный слой с входом n * m * m (n - кол-во каналов, m - размер изображения) и выходом 100, функция активации ReLU()
3. Линейный слой с выходом 10, ReLU()
4. Линейный аутпут слой. Учитывайте нашу задачу (см. флаг task) и используемую функцию потерь (criterion).


In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        set_random_seed(42) # - оставляйте эту строку для воспроизводимости результатов
        super().__init__()

        self.fc = nn.Sequential(
            # your code here
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
model = SimpleModel(in_channels=n_channels, num_classes=n_classes)
model = model.to(device)

# число эпох и lr можно менять
NUM_EPOCHS = 3
lr = 0.001

optimizer = optim.SGD(model.parameters(), lr=lr)

Используем сценарии train и test со встроенными метриками из семинара

In [ ]:
def train(model):
    print('==> Training ...')
    for epoch in range(NUM_EPOCHS):
      model.train()
      for inputs, targets in tqdm(train_loader):

          inputs = inputs.to(device)
          targets = targets.to(device)

          # forward + backward + optimize
          optimizer.zero_grad()
          outputs = model(inputs)

          if task == 'multi-label, binary-class':
              targets = targets.to(torch.float32)
              loss = criterion(outputs, targets)
          else:
              targets = targets.squeeze()
              loss = criterion(outputs, targets)

          loss.backward()
          optimizer.step()
    print('\n ==> Finished training')

In [ ]:
def test_split(model, split):
    model.eval()
    y_score = torch.tensor([]).to(device)

    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze()
                outputs = outputs.softmax(dim=-1)

            y_score = torch.cat((y_score, outputs), 0)

        y_score = y_score.cpu().detach().numpy()

        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)

        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))

def test(model):
    print('==> Evaluating ...')
    test_split(model, 'train')
    test_split(model, 'test')

In [ ]:
train(model)

In [ ]:
test(model)

# Задание 2. Модель посложнее
Создайте более сложную CNN модель по представленной ниже схеме и обучите ее
1. Конволюция с 6 каналами на выход, ядро 5x5 со страйдом 1 и 'same' паддингом (заполнение нулями), ReLU функция активации
2. AveragePooling2D, 2x2 ядро, страйд 2
3. Conv2D, 16 фичей, ядро 5x5 со страйдом 1 и без паддинга (padding='valid'), ReLU
4. AveragePooling2D, 2x2 ядро, страйд 2
5. Conv2D, 40 фичей, ядро 5x5 со страйдом 1 и без паддинга, ReLU
6. Flatten слой
7. Линейный слой со входом 40 * n * n  (n - размер картинки после предыдущих слоев) и выходом 60, ReLU
8. Dropout 50%
9. Линейный аутпут слой. Учитывайте нашу задачу (см. флаг task) и используемую функцию потерь (criterion)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        set_random_seed(42) # оставляем для воспроизводимости результатов
        super().__init__()

        # your code here

    def forward(self, x):
        # your code here
        return x

In [ ]:
model = CNNModel(in_channels=n_channels, num_classes=n_classes)
model = model.to(device)

# число эпох и lr можно менять
NUM_EPOCHS = 3
lr = 0.001

optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
train(model)

In [ ]:
test(model)

Сравните результаты обучения обеих сетей. Что изменилось в плане качества и изменилось ли? Сделайте выводы относительно созданных архитектур.

In [ ]:
# your conclusions here

# Задание 3. Своя модель
Попробуйте создать свою собственную архитектуру и попытайтесь максимально повысить качество предсказаний. Можно варьировать ЛЮБЫЕ параметры (кроме лосс функции, объявленной в начале) - число эпох, значение learning rate, оптимизатор, количество слоев (можно использовать дропауты, батч нормы и т.д.), функции активации и т.д.

За основу можно брать полученные на семинаре архитектуры



In [ ]:
class MyLittleModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        set_random_seed(42) # оставляем для воспроизводимости результатов
        super().__init__()

        # your code here

In [ ]:
model = MyLittleModel(in_channels=n_channels, num_classes=n_classes)
model = model.to(device)

# your code here

In [ ]:
train(model)

In [ ]:
test(model)

Сделайте вывод относительно ~вашего дитя~ вашей архитектуры. Поднялось ли качество, какие изменения повлияли сильнее всего?

In [ ]:
# your conclusions here

# Задание 4. Transfer learning
Идея Transfer learning: загрузим предобученную нейросеть и попробуем дообучить ее на наших данных. Дообучать будем только последний аутпут слой.



In [ ]:
from torchvision import models
from torchvision.models import ResNet50_Weights

In [ ]:
def get_resnet_model_finetune(num_classes, device, dropout=0):
        model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

        for param in model.parameters():
            param.requires_grad = False

        num_features = model.fc.in_features

        if dropout > 0:
            model.fc = nn.Sequential(
                nn.Dropout(dropout),
                nn.Linear(num_features, #your code here)
            )
        else:
            model.fc = nn.Linear(num_features, #your code here)

        model.params = model.fc.parameters()

        return model.to(device)

In [ ]:
model = get_resnet_model_finetune(n_classes, device)

Проверим, какой будет размерность выходных данных

In [ ]:
for X, y in train_loader:
    break

In [ ]:
model(X).shape

Рассчитаем качество модели без дообучения

In [ ]:
test(model)

Дообучим модель на нескольких эпохах

In [ ]:
model = get_resnet_model_finetune(n_classes, device)

NUM_EPOCHS = 3
lr = 0.001

optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
train(model)

In [ ]:
test(model)

Увеличим число эпох

In [ ]:
model = get_resnet_model_finetune(n_classes, device)

NUM_EPOCHS = 10
lr = 0.001

optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
train(model)

In [ ]:
test(model)

Больше эпох!

In [ ]:
model = get_resnet_model_finetune(n_classes, device)

NUM_EPOCHS = 30
lr = 3e-4

optimizer = optim.AdamW(model.parameters(), lr=lr)

In [ ]:
train(model)

In [ ]:
test(model)

Очень много эпох!! Так много, что можно пропустить, если GPU в бане

In [ ]:
model = get_resnet_model_finetune(n_classes, device)

NUM_EPOCHS = 100
lr = 3e-4

optimizer = optim.AdamW(model.parameters(), lr=lr)

In [ ]:
train(model)

In [ ]:
test(model)

Сделайте вывод касаемо применения Transfer learning с учетом результатов задания 3.

In [ ]:
# your conclusions here